## Classical Generator with Wasserstein style Discriminator

In [2]:
import numpy as np
import math
from tensorflow import keras as ks
import tensorflow as tf

import matplotlib.pyplot as plt

In [3]:
class DataGenerator():
    
    def __init__(self, n_points=500):
        mu_1 = 0.20
        sigma_1 = 0.03
        mu_2 = 0.40
        sigma_2 = 0.04
        self.n_points = n_points
        self.distribution = np.append(np.random.normal(mu_1, sigma_1, int(self.n_points / 2)), np.random.normal(mu_2, sigma_2, int(self.n_points / 2)))
    

In [4]:
class ClipConstraint(ks.constraints.Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return ks.backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}
    
# implementation of wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return ks.backend.mean(y_true * y_pred)

In [17]:
class WGAN():

    def __init__(self, n_epochs=50):
        super().__init__()
        # Initialize parameters
        self.latent_dim = 8
        
        # Training parameters
        self.n_epochs = n_epochs
        
        self.generator = self.build_generator()
        
        self.data_generator = DataGenerator()
        self.target_dist = self.data_generator.distribution
        
        self.critic = self.build_critic()
        
        # Build gan stack for fitting the generator
#         self.gan_stack = ks.Sequential()
#         self.gan_stack.add(self.decoding_model)
#         self.gan_stack.add(self.discriminator)
#         # TODO: Check learning rate
#         opt = ks.optimizers.RMSprop(lr=0.00005)
#         self.gan_stack.compile(loss=wasserstein_loss, optimizer=opt)
        self.gan_stack = self.build_gan()
        
    def build_generator(self):
        model = ks.Sequential()
        
        model.add(ks.layers.Dense(50, activation='elu', input_dim=self.latent_dim))
        model.add(ks.layers.Dense(1, activation='sigmoid', input_dim=self.latent_dim))
        
        return model
    
    def build_critic(self):
        # define the constraint
        const = ClipConstraint(0.01)
        
        model = ks.Sequential()
        model.add(ks.layers.Dense(50, activation='elu', input_shape=(1,), kernel_constraint=const))
#         model.add(ks.layers.Dense(100, activation='relu', kernel_constraint=const))
        model.add(ks.layers.Dense(50, activation='elu', kernel_constraint=const))
        model.add(ks.layers.Dense(1, activation='linear', kernel_constraint=const))
        
        # TODO: Investigate loss
        opt = ks.optimizers.RMSprop(lr=0.005)
        model.compile(loss=wasserstein_loss, optimizer=opt)
        return model
    
    def build_gan(self):
        model = ks.Sequential()
        
        model.add(self.generator)
        model.add(self.critic)
        
        opt = ks.optimizers.RMSprop(lr=0.00005)
        model.compile(loss=wasserstein_loss, optimizer=opt)
        return model

    # generate points in latent space as input for the generator
    def generate_latent_points(self, n_samples):
        # generate points in the latent space
        x_input = np.random.randn(self.latent_dim * n_samples)
        # reshape into a batch of inputs for the network
        x_input = x_input.reshape(n_samples, self.latent_dim)
        return x_input
    
    # use the generator to generate n fake examples, with class labels
    def generate_fake_samples(self, n_samples):
        # generate points in latent space
        x_input = self.generate_latent_points(n_samples)
        # predict outputs
        X = self.generator.predict(x_input)
        # create class labels with 1.0 for 'fake'
        y = np.ones((n_samples, 1))
        return X, y
    
    def train(self):
        self.critic.summary()
        self.generator.summary()
        
        # TODO: Resample real distribution continuously instead of using fixed amount of points
        X_real = np.stack(self.target_dist).reshape(len(self.target_dist), 1)
        y_real = -np.ones((len(self.target_dist), 1))
        
        n_critic = 5
        gen_batch_size = 32
        disc_batch_size = gen_batch_size * n_critic * 2
        
        for i in range(self.n_epochs):
            # Select read and fake data_points, we assume N_fake == N_real (Amount fake datapoints is equal to amount of real datapoints)
            idx = np.random.choice(np.arange(len(X_real)), int(disc_batch_size / 2))
            
            # Train on real
            X_real_sampled, y_real_sampled = np.take(X_real, idx), np.take(y_real, idx)
            c_loss1 = self.critic.train_on_batch(X_real_sampled, y_real_sampled)
            
            X_gan_fake = self.generate_fake_samples(int(disc_batch_size / 2))
            # create fake lables
            y_gan_fake = np.ones((int(disc_batch_size / 2), 1))
            
            # Train on fake
            c_loss2 = self.critic.train_on_batch(X_gan_fake, y_gan_fake)
            
            
            # Train generator stack
            # prepare points in latent space as input for the generator
            X_gan = self.generate_latent_points(gen_batch_size)
            # create inverted labels for the fake samples
            y_gan = -ones((gen_batch_size, 1))
            # update the generator via the critic's error
            self.discriminator.trainable = False
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            self.discriminator.trainable = True
            
            print("Iteration", i, "Gen_loss", g_loss, "C_loss1", c_loss1, "C_loss2", c_loss2)#, "Real accuracy", acc_real, "Fake accuracy", acc_fake)
#             self.generate_critic_belief(plot=True, iteration=i)
            if i % 5 == 0:
                self.compare_model_to_real()
                
    def generate_critic_belief(self, plot=False, iteration=-1):
        values = np.arange(0, 1, 0.01)
        beliefs = []
        for i in range(len(values)):
            beliefs.append(self.discriminator.predict([values[i]]))
        
        beliefs = np.reshape(beliefs, (100))
        
        if plot:
            plt.plot(np.arange(0, 1, 0.01), beliefs, label="belief")
            plt.title("Belief at iteration " + str(iteration))
#             plt.ylim(-1, 1)
            plt.show()
            
        return beliefs
            
    def compare_model_to_real(self):
        bins = np.linspace(0, 1, 100)
        
        plt.hist(self.target_dist, bins, density=True, label="True")
        predictions = []
        self.printProgressBar(iteration=0, total=10)
        j = 0
        for i in range(self.data_generator.n_points):
            predictions.append(self.generate_prediction())
            if i%(self.data_generator.n_points/10) == 0:
                j += 1
                self.printProgressBar(iteration=j, total=10)
#                 print(str(i/self.data_generator.n_points*100) + "%")
                
        predictions = np.stack(predictions)
        predictions = predictions.reshape((self.data_generator.n_points,))

        plt.hist(predictions, bins, density=True, label="False")
        
        plt.plot(np.arange(0, 1, 0.01), self.generate_critic_belief(), label="Critic belief")
        
        plt.legend()
        plt.show()
        
    def printProgressBar (self, iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
        # Print New Line on Complete
        if iteration == total: 
            print()
            
    
wgan = WGAN()
wgan.train()
# vqgan.generate_prediction()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 50)                100       
_________________________________________________________________
dense_56 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 51        
Total params: 2,701
Trainable params: 2,701
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 50)                450       
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 51        
Total params: 501
Trainable 

ValueError: in user code:

    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/mauk/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py: